In [54]:
import streamlit as st 
from scrapper import fk_scrapper_functions as sc
from scrapper import flipkart_json_scrapper_with_all_specifications as fk_scrapper
import time
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
competitor_data=pd.read_pickle('competitor_data1.pkl')

In [55]:
competitor_data=pd.read_pickle('competitor_data1.pkl')
st.write(competitor_data['all_specs'][0].get('Model Name'))
competitor_data['brand']=competitor_data['title'].str.split(" ").str[0]
brand_level_data=competitor_data.copy()
st.dataframe(brand_level_data)
for index, row in brand_level_data.iterrows():
    specs = row['all_specs']
    for key, value in specs.items():
        if key not in brand_level_data.columns:
            brand_level_data[key] = None
        brand_level_data.at[index, key] = value
most_important_feature={}
feature_name=[]
feature_count=[]
for column in brand_level_data.columns:
    feature_name.append(column)
    feature_count.append(brand_level_data[column].count())
most_important_feature['feature_name']=feature_name
most_important_feature['feature_count']=feature_count
important_features=pd.DataFrame(most_important_feature)
important_features['feautre_present_in_%_of_products']=important_features['feature_count']*100/important_features['feature_count'].max()
filtered_important_features=important_features[important_features['feautre_present_in_%_of_products'] >=10]['feature_name'].to_list()
final_scrapped_data=brand_level_data[filtered_important_features]
import analysis_function as f

2024-07-03 18:10:45.719 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert '' with type str: tried to convert to boolean", 'Conversion failed for column special_price with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.


In [61]:
final_scrapped_data[final_scrapped_data['final_selling_price']!='']

,fsn,title,rating,ratings_count,reviews_count,special_price,final_selling_price,mrp,flipkart_assured,Seller Name,highlights,paymentOptions,description,productDescription,specifications,fk_other_seller_info,reviews,productImagesCount,productVideosCount,image_link,all_specs,brand
0,SHOGHQ68THD9DKYD,"aadi Lightweight,Comfort,Summer,Trendy,Walking...",3.5,3088,147,False,299.0,999.0,False,Dadwanis Footwear,,,,[],{},{'Dadwanis Footwear': 299},"[{'rating': 5, 'review_header': 'Excellent', '...",6,0,https://rukminim1.flixcart.com/image/832/832/x...,{},aadi
1,SHOHFMMGHTTGXKWV,feetees Outdoors For Men Navy,0.0,0,0,False,1149.0,1999.0,True,FEETEESfootwear,,,,[],{},{'FEETEESfootwear': 1149},[],4,0,https://rukminim1.flixcart.com/image/832/832/x...,{},feetees
2,SHOGY9HUPGGN45ZZ,CLYMB Saga-1 Blue Walking/Outdoor/Gym & Tranin...,3.7,39,3,True,549.0,1499.0,True,ONUSMARKETING,,,,[],{},{'ONUSMARKETING': 549},"[{'rating': 5, 'review_header': 'Terrific purc...",6,0,https://rukminim1.flixcart.com/image/832/832/x...,{},CLYMB
3,SHOGZR69Q8BHWSNW,HRX by Hrithik Roshan URBAN CHUNKY Sneakers Fo...,4.1,5240,486,True,814.0,3699.0,True,HSAtlastradeFashion,,,,[],{},{'HSAtlastradeFashion': 814},"[{'rating': 5, 'review_header': 'Terrific purc...",7,0,https://rukminim1.flixcart.com/image/832/832/x...,{},HRX
4,SHOHYG3YCFDGHSAG,ATOM Supernova Sneakers For Men Yellow,4.2,77,4,False,1685.0,2895.0,True,Trusource,,,,[],{},{'Trusource': 1685},"[{'rating': 5, 'review_header': 'Excellent', '...",8,0,https://rukminim1.flixcart.com/image/832/832/x...,{},ATOM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,SHOGHJ5PT3Y9JGEG,"asian WNDR-13 Pro White Sports,Casual,Walking....",4.0,96371,12449,False,777.0,1499.0,False,Asianshoe,,,,[],{},"{'Asianshoe': 777, 'AsianFootwears': 777, 'PUN...","[{'rating': 5, 'review_header': 'Classy produc...",8,1,https://rukminim1.flixcart.com/image/832/832/x...,{},asian
116,SHOGYEYS8DBZXJHZ,"BRUTON Lite Sport Shoes , Runing Shoes Running...",3.9,833,73,True,646.0,2499.0,True,SALEMARTINDIA,,,,[],{},{'SALEMARTINDIA': 646},"[{'rating': 4, 'review_header': 'Very Good', '...",7,0,https://rukminim1.flixcart.com/image/832/832/x...,{},BRUTON
117,SHOGFXEPYRCRZT2G,VENDOZ Casual Stylish Trending Sneakers For Wo...,4.1,4751,294,False,629.0,999.0,True,REPOSEGROUP,,,,[],{},{'REPOSEGROUP': 629},"[{'rating': 5, 'review_header': 'Highly recomm...",6,1,https://rukminim1.flixcart.com/image/832/832/l...,{},VENDOZ
118,SHOGM856F7QYZFCQ,"BERSACHE Premium Sports ,Gym, Trending, Stylis...",4.1,3724,305,False,1357.0,4999.0,True,ORICUMFOOTWEAR,,,,[Lightweight Style : Make your feet feel light...,{},"{'BERSACHE': 1357, 'Vivaan': 1357, 'ORICUMFOOT...","[{'rating': 5, 'review_header': 'Great product...",9,1,https://rukminim1.flixcart.com/image/832/832/x...,{},BERSACHE


In [4]:
final_scrapped_data['ratings_count']=final_scrapped_data['ratings_count'].apply(lambda x: x if f.is_integer(x) else None).astype(float)

C:\Users\kashy\AppData\Local\Temp\ipykernel_7884\519216831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_scrapped_data['ratings_count']=final_scrapped_data['ratings_count'].apply(lambda x: x if f.is_integer(x) else None).astype(float)


In [18]:
rating_df=final_scrapped_data.groupby('brand').agg({"ratings_count":"sum"})
rating_df.reset_index(inplace=True)
all_discription_filter=list(set(filtered_important_features).difference(set(competitor_data.columns.tolist())))
list_of_brand=['Boult','boAt','Noise']
list_of_price=[899]
features_to_analyze=final_scrapped_data[final_scrapped_data['brand'].isin(list_of_brand) & final_scrapped_data['final_selling_price'].isin(list_of_price)]
all_discription_filter.extend(['brand','fsn'])
feature_df=features_to_analyze[all_discription_filter]
feature_df=feature_df.dropna(axis=1,how='all')
feature_df['Battery Life'].nunique()
features_after_not_unique=[description for description in feature_df if feature_df[description].nunique()>1]
feature_df=feature_df[features_after_not_unique]
feature_df.groupby(['brand',features_after_not_unique[2]]).agg({"fsn":["count",list]})

fsn                                                   
                     count                                               list
brand Headphone Type                                                         
Boult In the Ear         3  [ACCGEDKJF8WR4UWE, ACCGEDKJUCXSXVGK, ACCGEDKJJ...
      True Wireless     14  [ACCGRNMZUUHD5QFY, ACCGJT2ZEDUGFNMB, ACCGTBMG7...
Noise True Wireless      4  [ACCGQSGWDGQAVUM3, ACCGQSGWFMYHSMQV, ACCGQSGWF...
boAt  In the Ear         2               [ACCGJMG6RF3MJHDX, ACCGS9ZMAXHZFU2K]
      True Wireless      3  [ACCG48F2YZNGZ8D2, ACCFSDGXX3S6DVBG, ACCFSDGXT...

In [39]:
fe_distribution=pd.DataFrame(final_scrapped_data['Color'].value_counts())
fe_distribution.reset_index(inplace=True)
fe_distribution=fe_distribution.sort_values(by='count',ascending=False).head(10)
fe_fig=px.pie(fe_distribution,values='count',names='Color')

,fsn,title,rating,ratings_count,reviews_count,special_price,final_selling_price,mrp,flipkart_assured,Seller Name,highlights,paymentOptions,description,productDescription,specifications,fk_other_seller_info,reviews,productImagesCount,productVideosCount,image_link,all_specs,brand,Model Name,Color,Headphone Type,Inline Remote,Sales Package,Connectivity,Headphone Design,Accessories Included,Compatible Devices,Net Quantity,Sweat Proof,Foldable/Collapsible,Deep Bass,Water Resistant,Monaural,Open/Closed Back,Carrier Frequency,Magnet Type,Connector Plating,Connector Size,Controls,Other Features,Number of Pins,Boom Microphone,With Microphone,Sensitivity,Impedance,Signal to Noise Ratio,Modulation,Maximum Power Input,Minimum Frequency Response,Maximum Frequency Response,Noise Reduction,Other Sound Features,Audio Codec,Wireless Type,Wireless Range,Bluetooth Profiles,Bluetooth Version,Bluetooth Range,Headphone Power Source,Headphone Power Requirement,Headphone Battery,Power Supply,Battery Output,Battery Type,Battery Life,Battery Capacity,Charging Time,Play Time,Standby Time,Other Power Features,Warranty Summary,Warranty Service Type,Covered in Warranty,Not Covered in Warranty,Designed For,Series,System Requirements,Circumaural/Supraaural,Driver Type,Indicators,Theme,Technology Used,Headphone Driver Units,Cord Type,Transmitter Power Source,Width,Height,Depth,Weight,Cord Length,Other Dimensions,Domestic Warranty
